In [30]:
import json

data_path = 'data\\minecraft-data-3.63.0\\data\pc\\1.20\\'
files = ['biomes', 'blockCollisionShapes', 'blockLoot', 'blocks', 'effects', 'enchantments', 'entities', 'entityLoot', 'foods', 'instruments', 'items', 'language', 'loginPacket', 'materials', 'particles', 'protocol', 'recipes', 'tints', 'version']
minecraft_data = {}
md_foods = {}
 
# load the data
for file in files:
    with open(data_path + file + '.json') as f:
        minecraft_data[file] = json.load(f)

md_foods = {food_data['id']: food_data for food_data in minecraft_data['foods']}

artic_path = 'data\\ArticData\\'

In [31]:
# load datasets

# items
artic_items = {}
with open(artic_path + '1_20_items.json') as f:
    artic_items = json.load(f)
md_items = {item_data['id']: item_data for item_data in minecraft_data['items']}

# blocks
md_blocks = {block_data['id']: block_data for block_data in minecraft_data['blocks']}
artic_blocks = {}
with open(artic_path + '1_20_blocks.json') as f:
    artic_blocks = json.load(f)

# block loot
md_block_loot = {block_data['block']: block_data['drops'] for block_data in minecraft_data['blockLoot']}

# effects
artic_effects = {}
with open(artic_path + '1_20_potion_effects.json') as f:
    artic_effects = json.load(f)
md_effects = {effect_data['id']: effect_data for effect_data in minecraft_data['effects']}

# enchantments
artic_enchantments = {}
with open(artic_path + '1_20_enchantments.json') as f:
    artic_enchantments = json.load(f)
md_enchantments = {enchantment_data['id']: enchantment_data for enchantment_data in minecraft_data['enchantments']}

### Items

In [32]:
items = {}

for minecraft_id, artic in artic_items.items():
    item_id = artic['id']
    md = md_items[item_id]
    item = {}

    item['id'] = item_id
    item['minecraft_id'] = minecraft_id
    item['name'] = md['name']
    item['display_name'] = md['displayName']
    item['stack_size'] = md['stackSize']
    item['rarity'] = artic['rarity']
    item['fire_resistant'] = artic['fireResistant']
    
    if 'foodProperties' in artic:
        foodProperties = artic['foodProperties']
        effects = [{'id': artic_effects[effect['id']]['id'], # replace minecraft_id with effect_id
                    'duration': effect['duration'], 
                    'amplifier': effect['amplifier'], 
                    'chance': effect['chance']} for effect in foodProperties['effects']]
        
        # add food properties
        item['foodProperties'] = {
            'nutrition': foodProperties['nutrition'],
            'saturation_modifier': foodProperties['saturationModifier'],
            'effects': effects,
        }

    items[item_id] = item

### Blocks

In [33]:
# create a mapping of items by name
items_by_name = {item['name']: item for item in items.values()}

blocks = {}

# prepare block drops
def get_block_drops(drops):
    block_drops = []
    for drop in drops:
        silk_touch = 'optional'
        if 'silkTouch' in drop:
            silk_touch = 'required'
        elif 'noSilkTouch' in drop:
            silk_touch = 'forbidden'
        block_drops.append({
            'item': items_by_name[drop['item']]['id'], # use name mapping to get item id
            'min': drop['stackSizeRange'][0],
            'max': drop['stackSizeRange'][1],
            'chance': drop['dropChance'],
            'silk_touch': silk_touch
        })
    return block_drops

for minecraft_id, artic in artic_blocks.items():
    block_id = artic['id']
    md = md_blocks[block_id]
    block_name = md['name']

    block = {}
    block['block_id'] = block_id
    block['minecraft_id'] = minecraft_id

    # link to corresponding item
    if 'correspondingItem' in artic:
        corresponding_item_id = artic_items[artic['correspondingItem']]['id']
        block['corresponding_item'] = corresponding_item_id
        items[corresponding_item_id]['block_id'] = block_id

    block['display_name'] = md['displayName']
    block['hardness'] = md['hardness']
    block['explosion_resistance'] = artic['explosionResistance']
    block['diggable'] = md['diggable']
    block['gravity'] = artic['gravity']
    block['transparent'] = md['transparent']
    block['emit_light'] = md['emitLight']
    block['filter_light'] = md['filterLight']
    block['drops'] = []

    # some blocks have a base variant (wallSign -> sign, etc.)
    # drops are mapped to the base block only
    base_block_name = items[corresponding_item_id]['name'] # find base block
    if base_block_name in md_block_loot:
        md_drops = md_block_loot[base_block_name]
        block['drops'] = get_block_drops(md_drops)         # add base block drops

    # TODO does this mapping work for every blockLoot?

    blocks[block_id] = block

### Effects

In [34]:
effects = {}

for minecraft_id, artic in artic_effects.items():
    effect_id = artic['id']
    md = md_effects[effect_id]

    effect = {}
    effect['minecraft_id'] = minecraft_id
    effect['display_name'] = md['displayName']
    effect['type'] = md['type']
    effect['instantaneous'] = artic['instantaneous']

    effects[effect_id] = effect

### Enchantments

In [35]:
enchantments = {}

for minecraft_id, artic in artic_enchantments.items():
    enchantment_id = artic['id']
    md = md_enchantments[enchantment_id]

    enchantment = {}
    enchantment['minecraft_id'] = minecraft_id
    enchantment['display_name'] = md['displayName']
    enchantment['min_level'] = artic['minLevel']
    enchantment['max_level'] = artic['maxLevel']
    enchantment['rarity'] = artic['rarity']
    enchantment['category'] = artic['category']
    enchantment['weight'] = md['weight']
    enchantment['curse'] = artic['curse']
    enchantment['discoverable'] = artic['discoverable']
    enchantment['tradeable'] = artic['tradeable']
    enchantment['treasure_only'] = artic['treasureOnly']
    enchantment['exclude'] = [artic_enchantments[exclude]['id'] for exclude in artic['incompatibleEnchantments']]

    enchantments[enchantment_id] = enchantment

### Write to JSON

In [36]:
with open('test/blocks.json', 'w') as f:
    json.dump(blocks, f, indent=4)

with open('test/items.json', 'w') as f:
    json.dump(items, f, indent=4)

with open('test/effects.json', 'w') as f:
    json.dump(effects, f, indent=4)

with open('test/enchantments.json', 'w') as f:
    json.dump(enchantments, f, indent=4)